<a href="https://colab.research.google.com/github/YuriArduino/Estudos_Artificial_Intelligence/blob/Imers%C3%A3o-Agentes-de-IA---Alura/Imers%C3%A3o_Dev_Agentes_IA_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 Prepararando o ambiente

##1.2 Dependências

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00


##1.3 Imports

In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage
from pydantic import BaseModel, Field
from typing import Literal, List, Dict
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')
import sys
import textwrap
import time

##1.4 Conexão com o modelo

In [11]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=1.0, # Maior mais criativo, Menor mais objetivo
    )

###Para saber mais: Diferença **google-generativeai** X **langchain_google_genai**


*   **`google-generativeai` (a biblioteca nativa do Google)**: É como o **motor de um carro**. É a peça de engenharia principal, super poderosa, que faz o trabalho fundamental (gerar texto, contar tokens, etc.). Ela te dá acesso direto e total a todas as funcionalidades específicas daquele motor (o Gemini).

*   **LangChain (a biblioteca de orquestração)**: É como o **carro completo construído ao redor do motor**. Ele usa o motor (Gemini), mas adiciona o chassi, o volante, os pedais, o painel e os assentos. Ele torna o motor mais fácil de usar para um propósito maior (dirigir de um ponto A para um ponto B) e o conecta com outras partes (rodas, sistema de som, GPS).

Vamos detalhar as diferenças práticas:

---

### Tabela Comparativa

| Característica | `google-generativeai` (Nativa) | `langchain_google_genai` (Via LangChain) |
| :--- | :--- | :--- |
| **Propósito Principal** | Acesso direto e completo à API do Gemini. | Construir aplicações complexas com LLMs, orquestrando várias etapas. |
| **Nível de Abstração** | **Baixo.** Você interage diretamente com os conceitos da API do Google. | **Alto.** LangChain cria uma "camada de compatibilidade" sobre vários modelos. |
| **Funcionalidades** | Geração de conteúdo, contagem de tokens, embeddings, ajuste de segurança, etc. | **Tudo da nativa, e mais:** Chains, Agentes, Memória, RAG, etc. |
| **Flexibilidade de Modelo** | Feito exclusivamente para os modelos da família Gemini. | **Multi-modelo.** O código que você escreve pode ser facilmente adaptado para usar o GPT-4, Claude, etc. |
| **Facilidade (Tarefas Simples)** | Geralmente mais simples para uma única chamada de API. | Envolve um pouco mais de "boilerplate" (código de configuração) inicial. |
| **Facilidade (Tarefas Complexas)**| Você precisa construir toda a lógica (ex: memória de chat) do zero. | **Muito mais fácil.** Já fornece componentes prontos para tarefas complexas. |

---

### O que isso significa na prática?

#### 1. Abstração e Portabilidade

Com LangChain, o objeto `llm` que você criou é padronizado. Se amanhã você quisesse testar o modelo da OpenAI, você mudaria poucas linhas:

```python
# Com Google
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

# Com OpenAI (exemplo)
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

# O resto do seu código LangChain (chains, agentes) continuaria funcionando igual!
llm.invoke("Me conte uma piada.")
```

Se você usasse a biblioteca nativa, teria que reescrever todo o código de interação com o modelo.

#### 2. O Poder do Ecossistema LangChain

A verdadeira magia de usar LangChain, e o motivo pelo qual é ensinado na imersão, não é apenas para fazer uma pergunta ao modelo. É para construir sistemas mais complexos:

*   **Chains (Correntes):** Você pode criar sequências de tarefas. Por exemplo: "Passo 1: Pegue a pergunta do usuário. Passo 2: Traduza para o inglês. Passo 3: Envie ao Gemini. Passo 4: Pegue a resposta e traduza de volta para o português."
*   **Agentes (Agents):** Você pode dar ferramentas ao LLM. Por exemplo, dar a ele uma ferramenta de busca no Google. Se você perguntar "Qual a previsão do tempo para amanhã?", o agente pode decidir usar a ferramenta de busca, pegar o resultado e só então usar o Gemini para te dar uma resposta em linguagem natural.
*   **RAG (Retrieval-Augmented Generation):** É a técnica mais popular. Você pode fazer o Gemini "conversar com seus documentos". Você fornece uma base de dados (PDFs, sites, etc.), e o LangChain cuida de buscar a informação relevante nesses documentos para que o Gemini possa responder perguntas sobre eles.

### Conclusão

Para a imersão, ficar com **LangChain é a decisão certa**. Você não está apenas aprendendo a usar o Gemini, mas sim a **como construir aplicações robustas em torno de um LLM**, que é a habilidade mais valiosa no mercado hoje.

A contagem de tokens ser `llm.get_num_tokens()` em vez de `model.count_tokens()` é um pequeno sintoma dessa camada de abstração que o LangChain adiciona para tornar tudo mais padronizado e poderoso.

#2 Content





O resultado de `llm.invoke()` não é um texto puro, mas sim um **objeto**. No LangChain, esse objeto é chamado de `AIMessage`.

Este objeto `AIMessage` funciona como um "envelope" que contém várias informações sobre a resposta do modelo:

1.  **.content**: O conteúdo principal, ou seja, o texto da resposta que você queria. É o que está *dentro* do envelope.
2.  **.response_metadata**: Informações extras sobre a execução, como o motivo pelo qual o modelo parou de gerar texto (`finish_reason`) e as classificações de segurança (`safety_ratings`).
3.  **.usage_metadata**: Dados sobre o consumo de tokens na chamada (quantos tokens na entrada, quantos na saída e o total).
4.  **.id**: Um identificador único para aquela execução específica, útil para depuração.


### Comparando com o `StrOutputParser`

Agora você entende perfeitamente a diferença:

*   **Usar `.content`**: É a forma manual e direta de extrair o texto de uma **única** chamada. É perfeito para testes rápidos e quando você não está construindo uma sequência complexa de passos.

*   **Usar `| StrOutputParser()`**: É a forma automática de fazer a mesma coisa, mas de um jeito que se integra com o sistema de "correntes" (chains) do LangChain. Ele já "desempacota" o conteúdo para você e o entrega pronto para o próximo passo da sua chain.


In [12]:
resp_test = llm.invoke("Estamos Online?")
print(resp_test)

content='¡Sí! Estoy en línea y listo para ayudarte.\n\n¿En qué puedo asistirte hoy?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--8c3f49b7-c9d3-47b9-aad3-0de5ee8dcf4a-0' usage_metadata={'input_tokens': 4, 'output_tokens': 20, 'total_tokens': 751, 'input_token_details': {'cache_read': 0}}


In [13]:
#.content organiza o texto
resp_test = llm.invoke("Estamos Online?")
print(resp_test.content)

Sim, estamos online! Se você está lendo esta resposta, significa que a conexão está funcionando perfeitamente. 😊

Como posso te ajudar hoje?


#3 Tokens

Para um modelo de linguagem como o Gemini, os **tokens são os blocos de construção do texto**.

Um token não é exatamente uma palavra, uma letra ou uma sílaba. É a unidade fundamental que o modelo usa para processar e entender a linguagem.

*   Uma palavra curta como `"Eu"` pode ser **1 token**.
*   Uma palavra comum como `"gato"` pode ser **1 token**.
*   Uma palavra mais longa ou incomum como `"inteligência"` pode ser dividida em **2 ou mais tokens** (ex: `inteli` + `gência`).
*   Pontuação e espaços também contam e podem ser tokens.

**Exemplo Rápido:**
A frase `Olá, mundo!` é quebrada pelo modelo em algo como: `["Olá", ",", " mundo", "!"]` - resultando em **4 tokens**.

Em resumo, para o modelo, toda a nossa conversa (tanto o que enviamos quanto o que ele responde) é uma sequência de tokens.

### A Importância de um Contador de Tokens na nossa Imersão

No desenvolvimento de software, todo recurso é finito. Contar tokens é uma prática de **gerenciamento de recursos computacionais**. Ignorar essa métrica é como escrever um código que consome memória sem controle. A ferramenta `count_querys` que construímos é crucial por três razões técnicas:

**1. Gerenciamento de Custos e Carga de API:**
   Cada token processado representa uma unidade de computação que tem um custo. Em um ambiente de produção, esse custo é financeiro. Na Imersão, ele se reflete no consumo da nossa cota gratuita. O contador funciona como um *profiler*, permitindo-nos:
   *   Medir a "carga" exata que cada chamada impõe à API.
   *   Otimizar a comunicação para ser mais eficiente e evitar exceder os limites de requisições por minuto (*rate limits*).

**2. Respeitar a Janela de Contexto (Payload Capacity):**
   Todo modelo de linguagem opera com uma "janela de contexto" finita – a quantidade máxima de tokens que ele pode considerar em uma única requisição. Essa janela é a memória de trabalho da operação.
   *   Enviar um `payload` (dados de entrada) que exceda essa capacidade resultará em erro.
   *   Nosso contador é uma ferramenta de validação prévia, garantindo que nossas requisições sejam sempre viáveis e estejam dentro das especificações do modelo.

**3. Otimização de Prompts e Latência:**
   A engenharia de prompts é uma disciplina de otimização. Um prompt eficiente transmite a instrução máxima com o mínimo de tokens.
   *   **Sinal vs. Ruído:** Tokens irrelevantes são "ruído" que pode degradar a qualidade da resposta e aumentar a latência (tempo de resposta).
   *   Ao monitorar a contagem, aprendemos a construir prompts mais concisos e potentes, melhorando tanto a performance da aplicação quanto a precisão dos resultados.

---

## 3.1 Utilitário de Desenvolvimento: Profiler de Custo para API Gemini

In [ ]:
class count_querys:
    def __init__(self):
        self.total_tokens = 0
        print(" Count de querys ativo!")

    def processar_query(self, llm):
        """Processa uma query com count automático e formatação de saída."""
        query = input("Digite seu texto: ")

        if not query.strip():
            print("Nenhum texto inserido.")
            return

        # Envia para o LLM
        resposta = llm.invoke(query)

        # Conta os tokens desta query
        tokens_desta_msg = resposta.usage_metadata.get('total_tokens', 0) if resposta.usage_metadata else 0
        self.total_tokens += tokens_desta_msg

        # Formata a resposta para ter no máximo 90 caracteres por linha
        prefixo = " Resposta: "
        largura_maxima = 90

        # A função fill quebra o texto em linhas do tamanho desejado
        # e adiciona um recuo nas linhas seguintes para manter o alinhamento.
        texto_formatado = textwrap.fill(
            resposta.content,
            width=largura_maxima,
            initial_indent=prefixo,
            subsequent_indent=' ' * len(prefixo) # Recuo para alinhar com o início do texto
        )

        # Mostra a resposta da IA já formatada
        print(f"\n{texto_formatado}")

        # Mostra o count no final
        print(f"\n {tokens_desta_msg} tokens, total {self.total_tokens}")
# Crie o count uma vez
count = count_querys()

def query():
    global count
    if 'count' not in globals():
        count = count_querys()
    count.processar_query(llm)

# Status do count
def status():
    """Mostra o status atual dos tokens"""
    global count
    if 'count' not in globals():
        print("❌ Nenhuma conversa iniciada ainda. Use query() primeiro!")
    else:
        print(f" Total de tokens usados: {count.total_tokens}")

def reset():
    """Reseta o count de tokens"""
    global count
    count = count_querys()
    print(" count resetado!")

def help_chat():
  """Mostra as utilidades do chat."""
  print("""
UTILIDADES:
  query()     - Enviar mensagem
  status()     - Ver total tokens
  reset()      - Resetar count
  help_chat()  - Esta ajuda
""")

In [ ]:
query()

In [ ]:
status()

In [ ]:
reset()
print(status())

In [ ]:
help_chat()

#4 Estruturando a Saída com Pydantic



Para um programa de computador, processar variações  é propenso a erros. É importante uma estrutura de dados confiável, não um texto para interpretar.

### **Pydantic como um "Contrato" de Dados**

**Pydantic** é uma biblioteca Python para **validação de dados**. Ela nos permite definir "esquemas" ou "modelos" de dados usando classes Python. Em outras palavras, criamos um **contrato** que define a estrutura e os tipos de dados que esperamos receber.

O LangChain se integra perfeitamente com o Pydantic através do **`PydanticOutputParser`**. Este "parser" (analisador) faz duas coisas mágicas:

1.  **Instrui o LLM:** Ele analisa nosso modelo Pydantic e gera instruções claras de formatação que são enviadas ao LLM junto com nosso prompt.
2.  **Valida a Resposta:** Ele pega a resposta em texto do LLM, verifica se ela segue o "contrato", e a converte em um objeto Python real e validado.


###4.1 Configuração Inicial e Imports

In [ ]:
# Imports de bibliotecas de tipagem e validação
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

# Imports principais do LangChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage

# Configuração da API Key (maneira segura para o Colab)
import os
from google.colab import userdata

# Garanta que a variável de ambiente está configurada
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')

>Esta célula inicial é responsável por preparar nosso ambiente de trabalho.
>
>Importamos todas as bibliotecas necessárias, incluindo o Pydantic para definir nossa estrutura de dados e o LangChain para orquestrar a comunicação com o modelo Gemini.
>
>Também configuramos nossa chave de API do Google de forma segura usando os segredos do Colab.

##4.2 Definindo o "Cérebro" do Agente (O Prompt Principal)

In [ ]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

>Aqui definimos o SystemMessage, a personalidade e as diretrizes do nosso agente de IA.
>
>Este prompt é o "cérebro" da operação, explicando ao modelo seu papel, as regras de negócio para cada tipo de decisão e o que esperamos como resultado.

##4.3 Definindo o "Contrato" de Saída (O Esquema Pydantic)

In [ ]:
class TriagemOut(BaseModel):
    """Define a estrutura de saída esperada para a triagem de chamados."""

    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"] = Field(
        description="A decisão principal baseada na mensagem do usuário."
    )
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"] = Field(
        description="A urgência estimada do chamado."
    )
    campos_faltantes: List[str] = Field(
        default_factory=list,
        description="Uma lista de informações que o usuário precisa fornecer se a decisão for 'PEDIR_INFO'."
    )

>Esta é a parte central da validação. Usando Pydantic, criamos a classe TriagemOut que funciona como um "contrato".
>
>Ela força o LLM a gerar uma resposta que contenha exatamente os campos decisao, urgencia e campos_faltantes, com os tipos e valores que definimos. Se a IA gerar algo fora deste padrão, o LangChain acusará um erro.

##4.4 Célula Principal - Construindo e Executando a Lógica de Triagem

In [ ]:
# 1. Inicializa o LLM com temperatura 0.0 para respostas mais determinísticas
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",  # Usando um modelo padrão e eficiente para essa tarefa
    temperature=0.0
)

# 2. Cria a "chain" que conecta o LLM ao nosso esquema de saída Pydantic
# O método .with_structured_output() é o que faz a mágica acontecer!
triagem_chain = llm_triagem.with_structured_output(TriagemOut)

# 3. Define a função que invoca a chain e formata a saída
def triagem(mensagem: str) -> Dict:
    """
    Invoca a chain de triagem com a mensagem do usuário e retorna um dicionário.
    """
    # A chain recebe uma lista de mensagens (Sistema e Usuário)
    saida_pydantic: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    # .model_dump() converte o objeto Pydantic validado em um dicionário Python
    return saida_pydantic.model_dump()

>Esta é a célula principal onde a orquestração acontece. Nós:

1)   Instanciamos o modelo Gemini, configurando a temperature para 0.0 para obter resultados mais consistentes e previsíveis.
2)   Criamos a triagem_chain usando o poderoso método .with_structured_output(TriagemOut). É este comando que "ensina" o LLM a seguir nosso esquema Pydantic.
3)   Definimos a função triagem(), que encapsula a lógica de chamada, enviando o prompt do sistema e a mensagem do usuário para a chain.

##4.5 Teste e Validação

In [ ]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?",
          "Tenho uma dúvida sobre um benefício."]

for msg_teste in testes:
    print(f"Pergunta: '{msg_teste}'")
    resultado = triagem(msg_teste)
    print(f" -> Resposta Estruturada: {resultado}\n")

##4.6 **Teste e Validação(Com Controle de Taxa)**

> **Nota Importante sobre Limites de API (Rate Limiting)**
>
> Ao executar o loop `for` sem pausas, enviamos todas as nossas perguntas à API quase simultaneamente. Isso aciona um mecanismo de proteção da API do Google chamado **rate limiting**, resultando no erro `ResourceExhausted: 429`. Essencialmente, a API nos diz: "Por favor, vá com mais calma!".
>
> Para resolver isso, introduzimos o comando **`time.sleep(10)`** dentro do loop. Este comando pausa a execução por 10 segundos entre cada chamada, garantindo que respeitamos os limites da camada gratuita (ex: 2-5 requisições por minuto).
>
> Gerenciar a frequência das chamadas é uma **prática fundamental** no desenvolvimento com qualquer serviço externo, garantindo que nossa aplicação seja robusta e não seja bloqueada.

In [ ]:
# Importamos a biblioteca 'time' para adicionar pausas
import time

testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?",
          "Tenho uma dúvida sobre um benefício."]

for msg_teste in testes:
    print(f"Pergunta: '{msg_teste}'")
    resultado = triagem(msg_teste)
    print(f" -> Resposta Estruturada: {resultado}\n")

    # --- AJUSTE IMPORTANTE AQUI ---
    # Adiciona uma pausa de 10 segundos para não exceder o limite da API.
    # Para o free tier, um valor entre 10 e 20 segundos é seguro.
    print("...aguardando 10 segundos para a próxima requisição...\n")
    time.sleep(10)

print("✅ Todos os testes foram concluídos.")

>Com tudo configurado, esta célula executa um conjunto de testes para validar nosso sistema. Observamos como ele classifica diferentes tipos de perguntas, demonstrando a eficácia do prompt e da estruturação de saída com Pydantic.